<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/main/QnA_Llama_3_1_8b_with_Unsloth_D2_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 623, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 623 (delta 130), reused 58 (delta 58), pack-reused 444 (from 1)
Receiving objects: 100% (623/623), 11.68 MiB | 16.72 MiB/s, done.
Resolving deltas: 100% (331/331), done.


In [2]:
# %%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-jfhqhrh9/unsloth_74dd702ed8254cc4afe56e4154d6c802
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-jfhqhrh9/unsloth_74dd702ed8254cc4afe56e4154d6c802
  Resolved https://github.com/unslothai/unsloth.git to commit c247bfc72ce91b8150de5f7f0604c94fde147699
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.1 MB/s eta 0:00:00
   ━

In [3]:
# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

# **Setting Up the Environment**

In [4]:
!pip install rouge_score
from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c41900011c6284c53ae2c1b915162e4b6a98f6fd79d0e93c1d35930c59dbd509
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
# Importing Libraries

import os
import re
import pandas as pd # Data Handling
import torch
import tensorflow as tf
import numpy as np
import time
from google.colab import files
import shutil


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
from torch import __version__
from packaging.version import Version as V
if V(__version__) < V("2.4.0"):
  xformers = "xformers==0.0.27"
else:
  xformers =  "xformers"

In [8]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)


==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [58]:
tokenizer.eos_token = "<|endoftext|>"  # Set the EOS token if not done
tokenizer.bos_token = "<|beginoftext|>"  # Set the EOS token if not done
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

In [14]:
tokenizer.eos_token = "<|endoftext|>"
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<|endoftext|>'

# **Downloading 4-bit Quantized Model and Adding LoRA Adapters**

# **Dataset-1 :Preparing the Dataset for Fine-tuning**

In [16]:
D2_train_path = '/content/qM-AI-L/QA-System/Dataset-2/train.csv'
D2_test_path = '/content/qM-AI-L/QA-System/Dataset-2/test.csv'
D2_dev_path = '/content/qM-AI-L/QA-System/Dataset-2/dev.csv'


In [17]:
D2_train_df = pd.read_csv(D2_train_path)
D2_test_df = pd.read_csv(D2_test_path)
D2_dev_df = pd.read_csv(D2_dev_path)


In [18]:
D2_train_df.shape, D2_test_df.shape, D2_dev_df.shape

((1985, 2), (249, 2), (248, 2))

In [19]:
D2_train_df.columns,  D2_test_df.columns,  D2_dev_df.columns

(Index(['question', 'answer'], dtype='object'),
 Index(['question', 'answer'], dtype='object'),
 Index(['question', 'answer'], dtype='object'))

## **Add prompt format column to train df for model input**

In [60]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "answer for this question", # instruction
        "Is 9.11 larger than 9.9?", # input
        "", # output—leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 50)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
answer for this question

### Input:
Is 9.11 larger than 9.9?

### Response:
Yes

### Explanation:
Yes, 9.11 is larger than 9.9.<|end_of_text|>


In [61]:
#Function for Train dataset
EOS_TOKEN = tokenizer.eos_token
def formatted_train(x):

  instruction = "Generate an answer for the following question in less than three sentences."

  if x['question']:
    formatted_text = f""" {BOS_TOKEN} Below is an instruction that describes a task. \
    Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    {instruction}

    ### Input:
    {x['question']}

    ### Response:
    {x['answer']} {EOS_TOKEN}"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Response:
    {x['answer']} {EOS_TOKEN}"""

  return formatted_text

In [62]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatted_train(x):
#     instructions = "Generate an answer for the following question in less than three sentences."
#     inputs       = x['question']
#     outputs      = x['answer1']
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }

In [63]:
#Function for Test and Dev dataset
def format_test(x):
  instruction = "Generate an answer for the following question in less than three sentences."

  if x['question']:
    formatted_text = f""" {BOS_TOKEN} Below is an instruction that describes a task. \
    Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    {instruction}

    ### Input:
    {x['question']}

    ### Response:
     {EOS_TOKEN}"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Response:
     {EOS_TOKEN}"""

  return formatted_text

In [64]:
D2_train_df

,question,answer,formatted_text
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...,<|begin_of_text|> Below is an instruction tha...
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor...",<|begin_of_text|> Below is an instruction tha...
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...,<|begin_of_text|> Below is an instruction tha...
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...,<|begin_of_text|> Below is an instruction tha...
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...,<|begin_of_text|> Below is an instruction tha...
...,...,...,...
1980,What is the method of vectorization in Bag-of-...,The simplest method is to mark the presence of...,<|begin_of_text|> Below is an instruction tha...
1981,What is the significance of feature interactions?,Feature interactions capture relationships bet...,<|begin_of_text|> Below is an instruction tha...
1982,What is the difference between hard margin and...,Hard margin SVM assumes that the data is linea...,<|begin_of_text|> Below is an instruction tha...
1983,How does quantization error change with differ...,Quantization error decreases with larger data ...,<|begin_of_text|> Below is an instruction tha...


In [65]:
D2_train_df["formatted_text"] = D2_train_df.apply(formatted_train, axis=1)
D2_test_df["formatted_text"] = D2_test_df.apply(format_test, axis=1)
D2_dev_df["formatted_text"] = D2_dev_df.apply(format_test, axis=1)

**Check train prompt includes answer column/ data, and test and dev prompts does not include answers/targets**

In [66]:
D2_train_df

,question,answer,formatted_text
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...,<|begin_of_text|> Below is an instruction tha...
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor...",<|begin_of_text|> Below is an instruction tha...
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...,<|begin_of_text|> Below is an instruction tha...
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...,<|begin_of_text|> Below is an instruction tha...
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...,<|begin_of_text|> Below is an instruction tha...
...,...,...,...
1980,What is the method of vectorization in Bag-of-...,The simplest method is to mark the presence of...,<|begin_of_text|> Below is an instruction tha...
1981,What is the significance of feature interactions?,Feature interactions capture relationships bet...,<|begin_of_text|> Below is an instruction tha...
1982,What is the difference between hard margin and...,Hard margin SVM assumes that the data is linea...,<|begin_of_text|> Below is an instruction tha...
1983,How does quantization error change with differ...,Quantization error decreases with larger data ...,<|begin_of_text|> Below is an instruction tha...


In [67]:
D2_train_df['formatted_text'][1]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is a recurrent neural network (RNN)?\n\n    ### Response:\n    An RNN handles sequence data, capturing temporal dependencies with cycles in connections. <|endoftext|>'

In [68]:
D2_test_df['formatted_text'][0]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is feature selection?\n\n    ### Response:\n     <|endoftext|>'

In [69]:
D2_dev_df['formatted_text'][0]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    How do U-Nets skip connections contribute to better segmentation performance?\n\n    ### Response:\n     <|endoftext|>'

# **Convert DFs to Datasets**

In [70]:
!pip install datasets

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [72]:
from datasets import Dataset
D2_Train_dataset = Dataset.from_pandas(D2_train_df)
D2_Test_dataset = Dataset.from_pandas(D2_test_df)
D2_Dev_dataset = Dataset.from_pandas(D2_dev_df)

In [73]:
D2_Train_dataset, D2_Test_dataset, D2_Dev_dataset

(Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 1985
 }),
 Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 249
 }),
 Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 248
 }))

In [74]:
#check train dataset
D2_Train_dataset['formatted_text'][0:2]

[' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is the role of Siamese networks in domain adaptation?\n\n    ### Response:\n    Siamese networks can be used to align the representations of source and target domains by minimizing the discrepancy between them, making the model more robust to domain shifts. <|endoftext|>',
 ' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is a recurrent neural network (RNN)?\n\n    ### Response:\n 

# **Fine-tuning Google Gemma for Code Dataset**


In [75]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [76]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [77]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = D2_Train_dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = 1024,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1985 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [78]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,985 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.143400
2,1.213400
3,1.235100
4,1.216700
5,1.192300
6,1.072100
7,1.114200
8,1.078200
9,1.092900
10,1.277300


**#check test data for infrencing and answer generation**

In [79]:
from transformers import TextStreamer

In [80]:
D2_test_df['formatted_text'][2], D2_test_df['formatted_text'][3]

(' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is batch normalization and how does it work?\n\n    ### Response:\n     <|endoftext|>',
 ' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    How does Word2Vec create word embeddings?\n\n    ### Response:\n     <|endoftext|>')

# **Checkout the trained model generating answer**

In [ ]:
Prompt = D2_test_df['formatted_text'][45]


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    Prompt
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=50,    eos_token_id=tokenizer.eos_token_id,  # Set EOS token explicitly
    pad_token_id=tokenizer.pad_token_id,  # Ensure padding is handled
    do_sample=True)  # If sampling, or False for greedy search )
# Get the generated text by decoding the IDs
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text


# **Function to extract answer part from the model generated response**

In [82]:
generated_text

'  Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What are the components of a confusion matrix?\n\n    ### Response:\n     <|endoftext|> The confusion matrix is a table that summarizes the performance of a classification model on a dataset. It contains the true and predicted labels for each sample, along with the number of samples that fall into each category. The rows represent the actual labels, while the'

In [83]:

# **Function to extract answer part from the model generated response**
def get_answer(text):
    start_tag = "### Response:"

    # Find the start and end indices
    start_idx = text.find(start_tag)

    # Check if both tags are found
    if start_idx == -1:
        return None  # Tags not found

    # Extract content between the tags
    answer = text[start_idx + len(start_tag):].strip()

    return answer

get_answer(generated_text)

'<|endoftext|> The confusion matrix is a table that summarizes the performance of a classification model on a dataset. It contains the true and predicted labels for each sample, along with the number of samples that fall into each category. The rows represent the actual labels, while the'

In [84]:
D2_test_df.columns, D2_test_df.shape

(Index(['question', 'answer', 'formatted_text'], dtype='object'), (249, 3))

In [85]:
# prompt: slice D2_test_df to 100 rows randomly

D2_test_df_sample = D2_test_df.sample(n=20, random_state=42)


In [86]:
D2_test_df_sample['formatted_text'][6]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is variance in machine learning?\n\n    ### Response:\n     <|endoftext|>'

In [87]:
original_answers = []
llama_answers = []

for index, row in D2_test_df_sample.iterrows():
    original_answer = row['answer']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=50,     eos_token_id=tokenizer.eos_token_id,  # Set EOS token explicitly
    pad_token_id=tokenizer.pad_token_id,  # Ensure padding is handled
    do_sample=True)  # If sampling, or False for greedy search)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D2_Test_Llama_with_Unsloth_df = pd.DataFrame(zipped_subjects, columns = ['original_answers', 'Llama_answers'])
D2_Test_Llama_with_Unsloth_df

<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ### Input:
    How do word embeddings capture semantic relationships between words?

    ### Response:
     <|endoftext|> Word embeddings capture semantic relationships between words by representing words as vectors in a high-dimensional space, where the distance between two words is proportional to their semantic similarity. This allows for the computation of semantic relationships such as similarity, analogy, and entailment. <|
<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Gen

,original_answers,Llama_answers
0,Words with similar meanings are represented by...,<|endoftext|> Word embeddings capture semantic...
1,Variance refers to the error introduced by the...,<|endoftext|> <|endoftext|> <|endoftext|> <|en...
2,A cross-validation score is the performance me...,<|endoftext|> A cross-validation score is a me...
3,PEFT refers to techniques that allow fine-tuni...,<|endoftext|> Parameter-Efficient Fine-Tuning ...
4,Feature selection helps prevent overfitting by...,<|endoftext|> Feature selection is a technique...
5,A fully connected layer connects each neuron t...,<|endoftext|> A fully connected layer in deep ...
6,"GPT-3 is an advanced version of GPT-2, with 17...",<|endoftext|> GPT-3 is a large language model ...
7,TF-IDF is the product of TF and IDF.,<|endoftext|> TF-IDF is a statistical measure ...
8,Feature engineering for unstructured data invo...,<|endoftext|> Feature engineering is the proce...
9,Eigenvalues and eigenvectors are scalar values...,<|endoftext|> Eigenvalues are the values that...


In [88]:
# prompt: Using dataframe D1_Test_Gemma_with_Unsloth_df: get all rows which that are not None in Llama_answers

# Select rows where Llama_answers is not null
D2_Test_Llama3_with_Unsloth = D2_Test_Llama_with_Unsloth_df[D2_Test_Llama_with_Unsloth_df['Llama_answers'].notna()].reset_index()
D2_Test_Llama3_with_Unsloth


,index,original_answers,Llama_answers
0,0,Words with similar meanings are represented by...,<|endoftext|> Word embeddings capture semantic...
1,1,Variance refers to the error introduced by the...,<|endoftext|> <|endoftext|> <|endoftext|> <|en...
2,2,A cross-validation score is the performance me...,<|endoftext|> A cross-validation score is a me...
3,3,PEFT refers to techniques that allow fine-tuni...,<|endoftext|> Parameter-Efficient Fine-Tuning ...
4,4,Feature selection helps prevent overfitting by...,<|endoftext|> Feature selection is a technique...
5,5,A fully connected layer connects each neuron t...,<|endoftext|> A fully connected layer in deep ...
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",<|endoftext|> GPT-3 is a large language model ...
7,7,TF-IDF is the product of TF and IDF.,<|endoftext|> TF-IDF is a statistical measure ...
8,8,Feature engineering for unstructured data invo...,<|endoftext|> Feature engineering is the proce...
9,9,Eigenvalues and eigenvectors are scalar values...,<|endoftext|> Eigenvalues are the values that...


# ** Evaluate model performance- Rouge**

In [89]:
!pip install rouge-score  # Install the rouge-score library
from rouge_score import rouge_scorer

In [90]:
D2_test_scores = D2_Test_Llama3_with_Unsloth.copy()

In [91]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True) # Instantiate a RougeScorer object
    scores = scorer.score(reference, hypothesis) # Call the score method on the RougeScorer object
    return {metric: score.fmeasure for metric, score in scores.items()}

# Apply ROUGE score calculation to each row
D2_test_scores['rouge_scores'] = D2_test_scores.apply(lambda row: calculate_rouge(row['original_answers'], row['Llama_answers']), axis=1)

In [92]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores
0,0,Words with similar meanings are represented by...,<|endoftext|> Word embeddings capture semantic...,"{'rouge1': 0.31034482758620696, 'rouge2': 0.0,..."
1,1,Variance refers to the error introduced by the...,<|endoftext|> <|endoftext|> <|endoftext|> <|en...,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ..."
2,2,A cross-validation score is the performance me...,<|endoftext|> A cross-validation score is a me...,"{'rouge1': 0.3684210526315789, 'rouge2': 0.189..."
3,3,PEFT refers to techniques that allow fine-tuni...,<|endoftext|> Parameter-Efficient Fine-Tuning ...,"{'rouge1': 0.39436619718309857, 'rouge2': 0.05..."
4,4,Feature selection helps prevent overfitting by...,<|endoftext|> Feature selection is a technique...,"{'rouge1': 0.4225352112676057, 'rouge2': 0.202..."
5,5,A fully connected layer connects each neuron t...,<|endoftext|> A fully connected layer in deep ...,"{'rouge1': 0.4897959183673469, 'rouge2': 0.297..."
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",<|endoftext|> GPT-3 is a large language model ...,"{'rouge1': 0.32911392405063294, 'rouge2': 0.10..."
7,7,TF-IDF is the product of TF and IDF.,<|endoftext|> TF-IDF is a statistical measure ...,"{'rouge1': 0.25, 'rouge2': 0.08695652173913043..."
8,8,Feature engineering for unstructured data invo...,<|endoftext|> Feature engineering is the proce...,"{'rouge1': 0.4067796610169491, 'rouge2': 0.210..."
9,9,Eigenvalues and eigenvectors are scalar values...,<|endoftext|> Eigenvalues are the values that...,"{'rouge1': 0.3829787234042554, 'rouge2': 0.088..."


In [93]:
pd.json_normalize(D2_test_scores['rouge_scores'])

,rouge1,rouge2,rougeL,rougeLsum
0,0.310345,0.000000,0.172414,0.172414
1,0.000000,0.000000,0.000000,0.000000
2,0.368421,0.189189,0.263158,0.263158
3,0.394366,0.057971,0.225352,0.225352
4,0.422535,0.202899,0.225352,0.225352
5,0.489796,0.297872,0.408163,0.408163
6,0.329114,0.103896,0.202532,0.202532
7,0.250000,0.086957,0.250000,0.250000
8,0.406780,0.210526,0.372881,0.372881
9,0.382979,0.088889,0.297872,0.297872


In [94]:
D2_test_scores = pd.concat([D2_test_scores, pd.json_normalize(D2_test_scores['rouge_scores'])], axis=1)


In [95]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum
0,0,Words with similar meanings are represented by...,<|endoftext|> Word embeddings capture semantic...,"{'rouge1': 0.31034482758620696, 'rouge2': 0.0,...",0.310345,0.000000,0.172414,0.172414
1,1,Variance refers to the error introduced by the...,<|endoftext|> <|endoftext|> <|endoftext|> <|en...,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
2,2,A cross-validation score is the performance me...,<|endoftext|> A cross-validation score is a me...,"{'rouge1': 0.3684210526315789, 'rouge2': 0.189...",0.368421,0.189189,0.263158,0.263158
3,3,PEFT refers to techniques that allow fine-tuni...,<|endoftext|> Parameter-Efficient Fine-Tuning ...,"{'rouge1': 0.39436619718309857, 'rouge2': 0.05...",0.394366,0.057971,0.225352,0.225352
4,4,Feature selection helps prevent overfitting by...,<|endoftext|> Feature selection is a technique...,"{'rouge1': 0.4225352112676057, 'rouge2': 0.202...",0.422535,0.202899,0.225352,0.225352
5,5,A fully connected layer connects each neuron t...,<|endoftext|> A fully connected layer in deep ...,"{'rouge1': 0.4897959183673469, 'rouge2': 0.297...",0.489796,0.297872,0.408163,0.408163
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",<|endoftext|> GPT-3 is a large language model ...,"{'rouge1': 0.32911392405063294, 'rouge2': 0.10...",0.329114,0.103896,0.202532,0.202532
7,7,TF-IDF is the product of TF and IDF.,<|endoftext|> TF-IDF is a statistical measure ...,"{'rouge1': 0.25, 'rouge2': 0.08695652173913043...",0.250000,0.086957,0.250000,0.250000
8,8,Feature engineering for unstructured data invo...,<|endoftext|> Feature engineering is the proce...,"{'rouge1': 0.4067796610169491, 'rouge2': 0.210...",0.406780,0.210526,0.372881,0.372881
9,9,Eigenvalues and eigenvectors are scalar values...,<|endoftext|> Eigenvalues are the values that...,"{'rouge1': 0.3829787234042554, 'rouge2': 0.088...",0.382979,0.088889,0.297872,0.297872


In [96]:
!pip install nltk sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.6 MB/s eta 0:00:00


In [97]:
import sacrebleu
from nltk.translate import meteor_score
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [98]:
# Tokenization function
def tokenize(text):
    return text.lower().split()


In [99]:
def calculate_bleu(reference, hypothesis):
    # BLEU requires tokenized input for reference, but not for hypothesis
    reference = [tokenize(reference)]
    bleu = sacrebleu.corpus_bleu([hypothesis], reference) # hypothesis should be a string
    return bleu.score


D2_test_scores['bleu_score'] = D2_test_scores.apply(lambda row: calculate_bleu(row['original_answers'], row['Llama_answers']), axis=1)

# Calculate METEOR score
def calculate_meteor(reference, hypothesis):
    # Tokenize both reference and hypothesis
    tokenized_reference = tokenize(reference)
    tokenized_hypothesis = tokenize(hypothesis)
    return meteor_score.single_meteor_score(tokenized_reference, tokenized_hypothesis)  # Pass tokenized inputs

D2_test_scores['meteor_score'] = D2_test_scores.apply(lambda row: calculate_meteor(row['original_answers'], row['Llama_answers']), axis=1)

In [100]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum,bleu_score,meteor_score
0,0,Words with similar meanings are represented by...,<|endoftext|> Word embeddings capture semantic...,"{'rouge1': 0.31034482758620696, 'rouge2': 0.0,...",0.310345,0.000000,0.172414,0.172414,0.673588,0.205882
1,1,Variance refers to the error introduced by the...,<|endoftext|> <|endoftext|> <|endoftext|> <|en...,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,A cross-validation score is the performance me...,<|endoftext|> A cross-validation score is a me...,"{'rouge1': 0.3684210526315789, 'rouge2': 0.189...",0.368421,0.189189,0.263158,0.263158,0.673588,0.270274
3,3,PEFT refers to techniques that allow fine-tuni...,<|endoftext|> Parameter-Efficient Fine-Tuning ...,"{'rouge1': 0.39436619718309857, 'rouge2': 0.05...",0.394366,0.057971,0.225352,0.225352,0.000000,0.225355
4,4,Feature selection helps prevent overfitting by...,<|endoftext|> Feature selection is a technique...,"{'rouge1': 0.4225352112676057, 'rouge2': 0.202...",0.422535,0.202899,0.225352,0.225352,0.673588,0.516011
5,5,A fully connected layer connects each neuron t...,<|endoftext|> A fully connected layer in deep ...,"{'rouge1': 0.4897959183673469, 'rouge2': 0.297...",0.489796,0.297872,0.408163,0.408163,0.777275,0.536338
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",<|endoftext|> GPT-3 is a large language model ...,"{'rouge1': 0.32911392405063294, 'rouge2': 0.10...",0.329114,0.103896,0.202532,0.202532,0.000000,0.237726
7,7,TF-IDF is the product of TF and IDF.,<|endoftext|> TF-IDF is a statistical measure ...,"{'rouge1': 0.25, 'rouge2': 0.08695652173913043...",0.250000,0.086957,0.250000,0.250000,0.000000,0.292245
8,8,Feature engineering for unstructured data invo...,<|endoftext|> Feature engineering is the proce...,"{'rouge1': 0.4067796610169491, 'rouge2': 0.210...",0.406780,0.210526,0.372881,0.372881,0.000000,0.459959
9,9,Eigenvalues and eigenvectors are scalar values...,<|endoftext|> Eigenvalues are the values that...,"{'rouge1': 0.3829787234042554, 'rouge2': 0.088...",0.382979,0.088889,0.297872,0.297872,0.852245,0.307342


In [101]:
average_scores = {
    'average_rouge1': D2_test_scores['rouge1'].mean(),
    'average_rouge2': D2_test_scores['rouge2'].mean(),
    'average_rougeL': D2_test_scores['rougeL'].mean(),
    'average_rougeLsum': D2_test_scores['rougeLsum'].mean(),
    'average_bleu_score': D2_test_scores['bleu_score'].mean(),
    'average_meteor_score': D2_test_scores['meteor_score'].mean(),
}

In [102]:
print("Average Scores for Dataset-2 (Test-Answer-1) QnA task using LLaMA 8B_with_Unsloth model:\n")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")

Average Scores for Dataset-2 (Test-Answer-1) QnA task using LLaMA 8B_with_Unsloth model:

average_rouge1: 0.3332
average_rouge2: 0.1405
average_rougeL: 0.2645
average_rougeLsum: 0.2683
average_bleu_score: 0.3627
average_meteor_score: 0.3046


# **Repeat above evaluation process for the 2nd Answer annotation**

In [103]:
original_answers = []
llama_answers = []

for index, row in D1_test_df.iterrows():
    original_answer = row['answer2']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D1_Test_llama_with_Unsloth_df_A2 = pd.DataFrame(zipped_subjects, columns = ['answer-2', 'Llama_answers'])
D1_Test_llama_with_Unsloth_df_A2

NameError: name 'D1_test_df' is not defined

In [ ]:
D1_test_scores_A2 = D1_Test_llama_with_Unsloth_df_A2.copy()

In [ ]:
D1_test_scores_A2['rouge_scores'] = D1_test_scores_A2.apply(lambda row: calculate_rouge(row['answer-2'], row['Llama_answers']), axis=1)

In [ ]:
D1_test_scores_A2 = pd.concat([D1_test_scores_A2, pd.json_normalize(D1_test_scores_A2['rouge_scores'])], axis=1)

In [ ]:
D1_test_scores_A2

In [ ]:
D1_test_scores_A2['bleu_score'] = D1_test_scores_A2.apply(lambda row: calculate_bleu(row['answer-2'], row['Llama_answers']), axis=1)


D1_test_scores_A2['meteor_score'] = D1_test_scores_A2.apply(lambda row: calculate_meteor(row['answer-2'], row['Llama_answers']), axis=1)

In [ ]:
D1_test_scores_A2

In [ ]:
A2_average_scores = {
    'average_rouge1': D1_test_scores_A2['rouge1'].mean(),
    'average_rouge2': D1_test_scores_A2['rouge2'].mean(),
    'average_rougeL': D1_test_scores_A2['rougeL'].mean(),
    'average_rougeLsum': D1_test_scores_A2['rougeLsum'].mean(),
    'average_bleu_score': D1_test_scores_A2['bleu_score'].mean(),
    'average_meteor_score': D1_test_scores_A2['meteor_score'].mean(),
}

In [ ]:
print("Average Scores for Dataset-1 (Test-Answer-2) QnA task using Gemma_7b_with_Unsloth model:\n")
for metric, score in A2_average_scores.items():
    print(f"{metric}: {score:.4f}")


In [ ]:
# make a df with A2_average_scores and average_scores

# Create a DataFrame from the dictionaries
D1_Test_Scores = pd.DataFrame([average_scores, A2_average_scores], index=['Llama3-8b_model Vs.Answer-1', 'Llama3-8b_model Vs.Answer-2'])

# Transpose the DataFrame for better readability
D1_Test_Scores = D1_Test_Scores.T

D1_Test_Scores
